In this notebook, I have a series of cells containing different functions that produce different graphs using a csv file containing timestamps every 30 minutes from July 2022 to June 2023, with different readings such as solar generation, wind generation and battery storage.

The first thing I did was define the dataframe *df* using the provided csv file.

I have imported 5 libraries:


*   *pandas*, to read the data
*   *matplotlib.pyplot* and *matplotlib* to produce the graphs
*   *calendar*, to convert selected_month from a number to its corresponding month
*   *numpy* to define the x axes in my last function

I asked ChatGPT how I could produce a graph using an Excel Spreadsheet with dates and times and readings. ChatGPT responded telling me to import *pandas* and *matplotlib*.

Later on, I wanted to convert a number from 1-12 to the month name(e.g. 3 -> March), so I went on stackoverflow and found a solution that required importing *calendar*.

Finally, in my last function, I ran into an issue with my x axis, so I used numpy to define my x axis attributes.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import calendar
import numpy as np

df = pd.read_csv("/dbfs/FileStore/pidata_anon.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [0]:
#function not currently being used
def columns():
  columns_to_plot = []
  add_item = -1
  while True:
        add_item = input("Add a data reading of your choice. Enter '0' to exit.")
        if add_item == '0':
            break
        columns_to_plot.append(add_item)
  return columns_to_plot
#end of function

def month_reading(df, columns_to_plot, selected_month):


  df_filter = df[df['timestamp'].dt.month == selected_month]


  for column in columns_to_plot:
    plt.plot(df_filter['timestamp'], df_filter[column], label=column)

  plt.xlabel('Date and Time')
  plt.ylabel('Readings,MW')
  plt.title('Readings')
  plt.legend()
  plt.xticks(rotation=45)
  plt.show(block=True)



month_reading(df,['wind1'], 3)





1.The objective of the first function *month_reading()* is to produce a graph that takes a reading(s) and filters the dataframe to show readings from one selected month. For example, the graph can show solar generation from *pv1* across the month of July.


The program takes 3 arguments into the function *month_reading()*


*   *df* is the csv file
*   *columns_to_plot* is the reading(s) that the function will generate on the graph
*   *selected_month* is the month (1-12) that the function takes readings from df to produce a graph

When first asking ChatGPT to produce a graph, it told me how to plot the graph using the line;

>plt.plot(filtered_data['timestamp'], filtered_data[column], label=column)

which I then put within a for loop to loop through different elements in *columns_to_plot*.

ChatGPT also produced the following lines which format the graph's x and y axes as well as giving the graph a title and other attributes.

>plt.xlabel('Date and Time')  

>plt.ylabel('Readings,MW')

>plt.title('Readings')
  
>plt.legend()
  
>plt.xticks(rotation=45)
  
>plt.show(block=True)


I then ran into a problem where the data couldn't be read unless they were datetimelike values, so I asked ChatGPT to solve this issue, and it returned me this line;

>df['timestamp'] = pd.to_datetime(df['timestamp'])

Finally, I asked ChatGPT how to filter *df* to only give me readings for 1 month, so I was able to implement the following line;

> filtered_data = df[df['timestamp'].dt.month == selected_month]

After that, it was a matter of making sure the function takes the inputs correctly and producing the correct graph.


Using this function for *pv2* for example shows a line that goes up and down continuously. This indicates that the generation goes high during the day and low during night, which is correct because solar generation relies on sunlight.


I also made a function *columns()* which allows the user to select which readings they want the graph to show, but I decided to not use it for the time being because changing the argument myself is faster.

In [0]:
##df = pd.read_csv("/dbfs/FileStore/pidata_anon.csv")
##df['timestamp'] = pd.to_datetime(df['timestamp'])

def year_round_reading(df,columns_to_plot):
  df['month'] = df['timestamp'].dt.month
  mean_readings = df.groupby('month')[columns_to_plot].mean()

  mean_readings.plot(kind='line')
  plt.xlabel('Month')
  plt.ylabel('Mean Readings, MW')
  plt.title('Mean Readings for Each Month')
  plt.xticks(rotation=0)
  plt.show()

year_round_reading(df,['pv2'])

2.The objective of the second function *year_round_reading()* is to take an element *(columns_to_plot)* as an argument as well as *df* and show how the readings change from January to December.

This is a more useful representation of the data because not a lot will change throughout a month except the changes between day and night, but using the whole year can show us major differences that happen, typically in the winter months like January and Decemeber, and in the summer months, like June and July.

The most obvious indication of this is using the element *pv2*, which shows us that solar generation peaks at around 45MW in June, and drops to as low as 8MW in December, where the sunlight isn't as strong and daylight is shorter.

Firstly, I reused the code from my previous function to display the graph's axes and title, and changed the names according to what the graph is being used for.

I then had to create a new column for the month of the year that each reading belongs to;

>df['month'] = df['timestamp'].dt.month

that way I could take a mean reading for each month, group them together using *df.groupby()* and plot the graph.

>mean_readings = df.groupby('month')[columns_to_plot].mean()

In [0]:

def day_profile(df,selected_month,columns_to_plot):

    df['hour'] = df['timestamp'].dt.hour
    df_filter = df[df['timestamp'].dt.month == selected_month]
    hourly_avg_readings = df_filter.groupby('hour')[columns_to_plot].mean()
    hourly_avg_readings.plot(kind='line')



    plt.xlabel('Time')
    plt.ylabel('Mean Readings, MW')
    plt.title(f'Mean Day Profile for {calendar.month_name[selected_month]}')
    plt.xticks(range(24))
    plt.legend()
    plt.show()

day_profile(df,1,['pv2'])

3.The third function *day_profile()* shows the mean day profile for a selected month and has three arguments:
> *selected_month* that dictates which month's readings are used in the graph

> *columns_to_plot* which dictates which readings are plotted in the graph

>*df* - the dataframe

To make a day profile, I made a new column hour assigns each reading to its hour.

I then used the same logic from the first function to filter the dataframe to only the readings from the selected month.

Finally, I grouped the readings together using *groupby()* and plotted the graph with the same lines of code from other functions.

Using this function with *pv2* for July shows us that generation peaks around midday and drops to its lowest around midnight. This shows us that peak generation occurs when the sunlight is strongest, which is at midday, and lowest generation occurs when there is no sunlight, so during the night.

Using this function with *storage1* in January shows a high peak around 5pm. This means that the battery is exporting most of its energy around this time. This is because around 5 to 6pm is when a lot of people are using a lot of electricity through EVs and devices at home. Therefore, if the demand is high, the cost of electricity increases, so the batteries start exporting their energy. On the other hand, at 4am, the reading is at its lowest. The negative reading indicates that the batteries are storing energy. This is because at 4am, there is a much smaller demand for energy because most people are sleeping and not using any electricity. Therefore, the cost of electricity is low, so the batteries store the energy until the cost of electricity rises.

In [0]:
##df = pd.read_csv("/dbfs/FileStore/pidata_anon.csv")
##df['timestamp'] = pd.to_datetime(df['timestamp'])
def day_profile_12(df,columns_to_plot):    
    df['hour'] = df['timestamp'].dt.hour
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'cyan', 'magenta', 'brown', 'pink', 'gray','lime','black']
    for selected_month in range(1,13):
        df_filter = df[df['timestamp'].dt.month == selected_month]
        hourly_avg_readings = df_filter.groupby('hour')[columns_to_plot].mean()
        for column in columns_to_plot:
            plt.plot(hourly_avg_readings.index, hourly_avg_readings[column], color = colors[selected_month-1] , label=f'{calendar.month_name[selected_month]}')
        
    plt.xlabel('Time')
    plt.ylabel(f'Mean {column} ,MW')
    plt.title(f'Mean {column} day profile for each month')
    plt.xticks(range(24))
    plt.legend(loc='center', bbox_to_anchor=(0.5, -0.3), frameon=True, ncol = 4)
    plt.show()

day_profile_12(df,['wind1'])

4.The fourth function *day_profile_12()* is similar to the previous function, but displays every single month onto 1 graph, rather than just 1 selected month.

The function has 2 arguments; *columns_to_plot()* which the function uses to plot the graph with the chosen reading(s), and *df*.

The code is very similar to the previous function, but I incorporated a for loop, that loops selected_month from 1 to 12(inclusive) and produces a plot for each month.

I also customised the legend to be easier to read and not obstruct the graph, which was done with this line;

>plt.legend(loc='center', bbox_to_anchor=(0.5, -0.3), frameon=True, ncol = 4)

matplotlib only uses 10 colours by default, and I wanted to have each line be a different colour, so I made an array *colors* that uses the *selected_month* counter to loop through the array and use a different colour for each line. I also made sure to -1 from the counter when using *colors* because *selected_month* starts from index 1 whilst colors starts from index 0.

Using the element *pv2* we can see that the months June and July have the highest peaks, indicating that the sunlight is strongest in those months, so generation is at its highest. Furthermore, the same months reach close to 0MW at a later time (7-8pm) indicating that the sunlight is not only stronger, but lasts longer. This shows us that solar generation is highest in summer because of the stronger sunlight and longer days. We can also see December has by far the lowest peak reading, which suggests that the winter has the weakest sunlight, and the reading also drops to near 0MW much earlier(around 3pm), which suggests that the day is much shorter in winter than summer. However, regardless of the time of year, the peak reading always seems to be around 11am-12pm and the readings drop to near 0MW during the night.

We can also plot storage1, which shows a similar pattern to the previous function. This time round, the months seem to be closer to one another, however the winter months have stronger spikes in the early morning and late afternoon. This is because there is less solar generation in the winter months and the temperatures are colder, so costs of electricity are higher because demand is higher and generation is smaller. Therefore, the batteries store more energy in the night and release the energy in the afternoon. However, in summer, the costs of electricity don't change as much, so farms are less incentivised to store and sell energy to make profits.

In [0]:
def day_profile_subplots(df,columns_to_plot):    
    df['hour'] = df['timestamp'].dt.hour
    #df['hh'] = df['timestamp'].dt.strftime("%H%M")
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'cyan', 'magenta', 'brown', 'pink', 'gray','lime','black']

    fig,axes = plt.subplots(3,4, figsize=(20, 15), sharex=False,sharey=True)
    axes = axes.flatten()
    plt.subplots_adjust(hspace=0.5, wspace=0.3)


    for selected_month in range(1,13):
        df_filter = df[df['timestamp'].dt.month == selected_month]
        hourly_avg_readings = df_filter.groupby('hour')[columns_to_plot].mean()
        for column in columns_to_plot:
            ax = axes[selected_month - 1]
            ax.plot(hourly_avg_readings.index, hourly_avg_readings[column], color = colors[selected_month-1] , label=f'{calendar.month_name[selected_month]}')
            ax.set_title(f'{columns_to_plot} - {calendar.month_name[selected_month]}')
            ax.set_xlabel('Hour of the Day')
            ax.set_ylabel('Average Readings, MW')
            ax.set_xticks([0, 6, 12, 18])
            # ax.set_ylim(-5,5)
            ax.yaxis.set_tick_params(labelbottom=True)
            ax.legend()

    fig.suptitle("Day Profile for each month")

day_profile_subplots(df,['wind1'])

5.The fifth function *day_profile_subplots()* does the same thing as the previous function but rather than displaying 12 plots on one graph, each plot has its own graph in  a 4x3 formation. Once again, the function has the arguments *columns_to_plot* and *df*

I first asked ChatGPT on how I could produce 12 graphs in a 4x3 formation, and I got a response saying to use the *subplots()* function. Specifically, ChatGPT gave me these 3 lines;

>fig,axes = plt.subplots(3,4, figsize=(20, 15), sharex=False,sharey=True)

>axes = axes.flatten()

>plt.subplots_adjust(hspace=0.5, wspace=0.3)

The first line specifies the size and dimensions of the graphs and the attributes of the axes. The next line converts axes from a 2D to 1D list, which makes it easier to loop through. The third line changes the spacing horizontally and vertically between each graph.

Chat GPT also gave me the following line that makes sure there isn't an index error;

>ax = axes[selected_month - 1]

In previous functions, I used *plt.* for lines of code responsible for producing each graph. However, now that I'm producing several graphs in one execution, ChatGPT informed me to use *ax.* which would ensure that each iteration of the for loop produces a different graph.

I wanted the y axis to have the same ticks (i.e 0,10,20,30,40), which *sharey = True* does, but only the 3 graphs on the left hand side would have labelled ticks. This line made sure every graph had labelled ticks while ensuring all the ticks were the same.

>ax.yaxis.set_tick_params(labelbottom=True)

Using *fig.suptitle()*, I gave the subplots a title too.

With this function, we can obtain the same conclusions from the previous function, such as solar generation being higher in the summer months, but this graph makes each individual month easier to read and understand.

In [0]:
def day_profile_everyday(df,selected_month,columns_to_plot):

    df['hour'] = df['timestamp'].dt.hour
    df['day'] = df['timestamp'].dt.day
   
    df_filter_month = df[df['timestamp'].dt.month == selected_month]

    for day_number in range(1, (calendar.monthrange(2023,selected_month)[1])+1):
        df_filter_day = df_filter_month[df_filter_month['timestamp'].dt.day == day_number]
        for column in columns_to_plot:
            plt.plot(np.linspace(0,23.5,num= 48),df_filter_day[column],alpha = 0.35,color = 'grey',linewidth = 0.75)

    hourly_avg_readings = df_filter_month.groupby('hour')[columns_to_plot].mean()
    hourly_avg_readings.plot(kind='line',linewidth = 4, ax=plt.gca())

    plt.xlabel('Time')
    plt.ylabel('Mean Readings, MW')
    plt.title(f'Mean Day Profile for {calendar.month_name[selected_month]}')
    plt.xticks(range(24))
    #plt.xlim([6,18])
    plt.legend()
    plt.show()

day_profile_everyday(df,11,['wind1'])

6.This final function, *day_profile_everyday()* shows an opaque line plot for every day's day profile within a month, and then also displays a thicker line that shows a mean of all the other lines.
The function has 3 arguments:

*  *df*
*  *selected_month*
*  *columns_to_plot*

In order to plot 28-31 lines, I used a for loop, looping from 1 to however many days were in the selected month, which I did using *calendar.monthrange(2023,selected_month)[1]* to make sure the loop only iterates the exact amount of times it needs to, regardless of the month.

To loop through each day, I filtered *df* to only the required month in *df_filter_month*, and then in each iteration , I filtered *df_filter_month* to only the readings from the required day into *df_filter_day*.

To make the day profile lines less significant in appearance, I used *alpha = 0.35* and *linewidth = 0.75* to make the lines more opaque and thinner respectively. Similarly, to make the mean line more visible, I made the line width much higher.

I then had to import *numpy* and use *numpy.linspace(0,23.5,num = 48)* in order to properly define the x axis attributes.

*  *0* is the first value
*  *23.5* is the last value
*  *num = 48* is the number of readings (2 per hour)

Finally, because I plotted the 28-31 lines seperately from the mean line, they shared different axes, and thus were plotted on seperate graphs. Therefore, to make sure they had the same axes and could be plotted in one graph together, I used *ax=plt.gca()* where *gca* means 'get current axis'.

Using this function for *pv2* or *storage1* for example shows us that day to day the trend is very similar and not much changes within a single month. However, if we plot wind1, we can see that wind generation is much less predictable as it is much more dependant on environmental factors, so the wind forecasts may be less predictable and insightful.